In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate
import Utils
import cv2
from facenet_pytorch import MTCNN
from Weights import *

In [ ]:
data_root = '../../data/'
def filtered_labels(path, columns = ['skin_tone','gender','age']):
    df = pd.read_csv(data_root + path)
    if columns is not None and len(columns) > 0:
        df = df.dropna(how='any',subset=columns)
        
    df['skin_tone'] = df.skin_tone.apply(lambda x: int(x.replace('monk_',''))-1)
    df['gender'] = df.gender.apply(lambda x: int(x == 'male'))
    age_map = {
        '0_17': 0,
        '18_30': 1,
        '31_60': 2,
        '61_100': 3,
    }
    df['age'] = df.age.apply(lambda x: age_map[x])
    
    return df

train_labels = filtered_labels('train_labels.csv')
test_labels = filtered_labels('labels.csv')

In [ ]:
def detect_nonfaces(df,root,image_size=256):
    mtcnn = MTCNN(image_size = image_size)
    def is_face(file):
        img = Image.open(root+file)
        try:
            face = mtcnn(img)
            if face is None:
                return False
            return True
        except Exception as e:
            print(e,file)
            return True
        
    df['is_face'] = df.name.apply(is_face)
    return df

train_labels = detect_nonfaces(train_labels,data_root)
test_labels = detect_nonfaces(test_labels,data_root)
train_labels[train_labels.is_face]

In [ ]:
validation_labels = train_labels.sample(frac=.2,replace=False)
train_labels = train_labels.drop(validation_labels.index)
train_labels

In [ ]:
train_labels.to_csv('train_data_clean.csv',index=False)
validation_labels.to_csv('validation_data_clean.csv',index=False)
test_labels.to_csv('test_data_clean.csv',index=False)